In [3]:
import csv
from openai import OpenAI
from typing import List

# задайте значение API ключа, полученный в OpenRouter
OPENROUTER_API_KEY = "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"

def read_batches(file_path: str, batch_size: int = 50) -> List[List[str]]:
    """Чтение файла батчами по указанному количеству строк"""
    batches = []
    with open(file_path, 'r', encoding='utf-8') as f:
        batch = []
        for line in f:
            cleaned_line = line.strip()
            if cleaned_line:
                batch.append(cleaned_line)
                if len(batch) >= batch_size:
                    batches.append(batch)
                    batch = []
        if batch:  # Добавляем последний неполный батч
            batches.append(batch)
    return batches

def get_api_response(batch: List[str]) -> str:
    """Отправка запроса к API и получение ответа"""
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
    )

    prompt = f"""
**Задача:** Классифицируйте текст по следующим категориям и предоставьте результат в CSV-формате.

---

### **Категории и подкатегории**
1. **HATE** (Дискриминация/ненависть к защищенным группам):
   - Подкатегории: RACE, RELIGION, GENDER, ETHNICITY, LGBTQ+, DISABILITY.
   - Примеры: расистские высказывания, гомофобные комментарии, сексизм.

2. **OFFENSE** (Оскорбления без группового контекста):
   - Подкатегории: PERSONAL_INSULT, PROFANITY, DEROGATORY_LANGUAGE.
   - Примеры: личные оскорбления, нецензурная брань, унизительные выражения.

3. **VIOLENCE** (Угрозы, призывы к насилию):
   - Подкатегории: THREAT, GLORIFICATION, INSTRUCTION.
   - Примеры: прямые угрозы физической расправы, романтизация терроризма.

4. **MISINFORMATION** (Дезинформация/фейки):
   - Подкатегории: CONSPIRACY, FAKE_NEWS, PSEUDOSCIENCE.
   - Примеры: отрицание Холокоста, теории заговора, антивакцинные мифы.

5. **NEUTRAL** (Факты, рассуждения без эмоциональной окраски):
   - Подкатегории: FACT, OPINION, HYPOTHETICAL.
   - Примеры: научные утверждения, философские вопросы.

6. **IGNORE** (Незначимый текст):
   - Применяется для: междометий, бессмысленных фраз, коротких слов без контекста ("хотеть", "ага").

---

### **Степень серьезности (Severity)**
- **HIGH**: Прямые угрозы, жесткая дискриминация, опасная дезинформация.
- **MEDIUM**: Косвенные оскорбления, недоказанные теории.
- **LOW**: Сарказм без злого умысла, легкая ненормативная лексика.

---

### **Инструкции**
1. **Пропускайте строки с категорией `IGNORE`** (не включайте их в CSV).
2. Для **NEUTRAL** обязательно указывайте подкатегорию и severity=LOW.
3. Если подкатегория неочевидна — используйте `GENERIC`.
4. Избегайте "переклассификации". Например, сарказм без угроз → `NEUTRAL.OPINION`, а не `OFFENSE`.

---

### **Формат вывода**
CSV с колонками: `text`, `category`, `subcategory`, `severity`.

**Пример вывода:**
```csv
text,category,subcategory,severity
"Да кому ты пиздишь))...",OFFENSE,PERSONAL_INSULT,MEDIUM
"Ядерный конфликт между...",VIOLENCE,THREAT,HIGH
"520 atmospheric nuclear...",MISINFORMATION,CONSPIRACY,MEDIUM

    Тексты для классификации:
    """ + "\n".join([f'"{text}"' for text in batch])

    try:
        completion = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "gg",
                "X-Title": "Text Classification",
            },
            model="deepseek/deepseek-chat-v3-0324:free",
            messages=[{"role": "user", "content": prompt}]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"API Error: {str(e)}")
        return ""


def process_file(input_path: str, output_path: str):
    """Основной процесс обработки файла"""
    batches = read_batches(input_path)

    with open(output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["text", "category", "subcategory", "severity"])

        for i, batch in enumerate(batches):
            if i < 150:
                continue
            
            print(f"Processing batch {i+1}/{len(batches)}")
            response = get_api_response(batch)

            if response:
                try:
                    # Парсинг CSV ответа
                    lines = response.strip().split('\n')
                    reader = csv.reader(lines[1:])  # Пропускаем заголовок
                    for row in reader:
                        if len(row) == 4 and row[1] != "IGNORE":
                            writer.writerow(row)
                except Exception as e:
                    print(f"Error parsing response: {str(e)}")

if __name__ == "__main__":
    input = "b.txt"
    output = "b_razmech_v3_150.csv"
    process_file(input, output)
    print(f"Classification complete. Results saved to {output}")

Processing batch 151/384
Processing batch 152/384
Processing batch 153/384
Processing batch 154/384
Processing batch 155/384
Processing batch 156/384
Processing batch 157/384
Processing batch 158/384
Processing batch 159/384
Processing batch 160/384
Processing batch 161/384
Processing batch 162/384
Processing batch 163/384
Processing batch 164/384
Processing batch 165/384
Processing batch 166/384
Processing batch 167/384
Processing batch 168/384
Processing batch 169/384
Processing batch 170/384
Processing batch 171/384
Processing batch 172/384


KeyboardInterrupt: 